# Machine learning for sentiment classification on movie reviews


In [1]:
# load common librairies
#import numpy as np
import pandas as pd

## Load and prepare data

In [41]:
# preprocessing: prepare data
col_names = ['content', 'label']
pos = pd.DataFrame(columns = col_names)
neg = pd.DataFrame(columns = col_names)

# on va dans le dossier pos
# pour chaque contenu de fichier, on le met dans une cellule de DataFrame
import os
for fend in os.listdir('./dataset1/pos/'):
#    print(str('./dataset1/pos/'+fend))
#    data = pd.read_csv('./dataset1/pos/'+fend, sep = None, header = None)
#    print(data)
    file = open('./dataset1/pos/'+fend, 'r')
    data = file.read()
    file.close()
    pos = pos.append(pd.DataFrame({'content':[data], 'label':[1]}))
# idem pour neg
for fend in os.listdir('./dataset1/neg/'):
    #data = pd.read_csv('./dataset1/neg/'+fend, sep = None, header = None)
    file = open('./dataset1/neg/'+fend, 'r')
    data = file.read()
    file.close()
    neg = neg.append(pd.DataFrame({'content':[data],'label':[-1]}))
print('done')
print('number of positive samples: {} '.format(len(pos)))
print('number of negative samples: {} '.format(len(neg)))

done
number of positive samples: 1000 
number of negative samples: 1000 


In [44]:
# examine the first 10 rows
pos.head(10)

,content,label
0,films adapted from comic books have had plenty...,1
0,every now and then a movie comes along from a ...,1
0,you've got mail works alot better than it dese...,1
0,""" jaws "" is a rare film that grabs your atten...",1
0,moviemaking is a lot like being the general ma...,1
0,"on june 30 , 1960 , a self-taught , idealistic...",1
0,"apparently , director tony kaye had a major ba...",1
0,one of my colleagues was surprised when i told...,1
0,"after bloody clashes and independence won , lu...",1
0,the american action film has been slowly drown...,1


In [45]:
neg.head(10)

,content,label
0,"plot : two teen couples go to a church party ,...",-1
0,the happy bastard's quick movie review \ndamn ...,-1
0,it is movies like these that make a jaded movi...,-1
0,""" quest for camelot "" is warner bros . ' firs...",-1
0,synopsis : a mentally unstable man undergoing ...,-1
0,capsule : in 2176 on the planet mars police ta...,-1
0,"so ask yourself what "" 8mm "" ( "" eight millime...",-1
0,that's exactly how long the movie felt to me ....,-1
0,call it a road trip for the walking wounded . ...,-1
0,plot : a young french boy sees his parents kil...,-1


In [48]:
# concat positive and negative samples
reviews = pos.append(neg)
print(reviews.head(10))
print(reviews.tail(10))

                                             content label
0  films adapted from comic books have had plenty...     1
0  every now and then a movie comes along from a ...     1
0  you've got mail works alot better than it dese...     1
0   " jaws " is a rare film that grabs your atten...     1
0  moviemaking is a lot like being the general ma...     1
0  on june 30 , 1960 , a self-taught , idealistic...     1
0  apparently , director tony kaye had a major ba...     1
0  one of my colleagues was surprised when i told...     1
0  after bloody clashes and independence won , lu...     1
0  the american action film has been slowly drown...     1
                                             content label
0  man , this was one wierd movie . \nsimilar to ...    -1
0  the king and i , a warner brothers animated , ...    -1
0  synopsis : cro-magnon ayla loses her mother to...    -1
0  salaries of hollywood top actors are getting o...    -1
0  movies like six days , seven nights make me ma...    

In [49]:
# define X and y for use with COUNTVECTORIZER
X = reviews.content
y = reviews.label
print(X.shape)
print(y.shape)

(2000,)
(2000,)


In [60]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.25)
print('number of training samples: {} '.format(len(X_train)))
print('number of training samples: {} '.format(len(X_test)))
print([X_train.head(10), y_train.head(10)])
print([X_test.head(10), y_test.head(10)])

number of training samples: 1500 
number of training samples: 500 
[0    james cmaeron's breakthrough feature was the f...
0    this is not a simple plan about finding a plan...
0    the seasoned capt . dudley smith ( james cromw...
0    in `enemy at the gates' , jude law is a gifted...
0    call me crazy , but i don't see saving private...
0    writer/director lawrence kasdan had a hand in ...
0    john boorman's " zardoz " is a goofy cinematic...
0    it is an understood passion and an understood ...
0    seen april 16 , 1999 at 10 p . m . at crossgat...
0    american pie 2 is filled with laughs . \nbut t...
Name: content, dtype: object, 0     1
0    -1
0     1
0     1
0     1
0    -1
0    -1
0     1
0     1
0    -1
Name: label, dtype: object]
[0    look back at all the times in your life when t...
0    okay , i just don't know why , but i seem to b...
0    the premise of turbulence is i'm sure very fam...
0    synopsis : al simmons , top-notch assasin with...
0    the rich legacy of

## Representing text as numerical data

In [61]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [62]:
# learn the "vocabulary" of the training data (occurs in-place)
vect.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [63]:
# examine the fitted vocabulary
vect.get_feature_names()

['00',
 '000',
 '0009f',
 '007',
 '00s',
 '03',
 '04',
 '05',
 '05425',
 '10',
 '100',
 '1000',
 '10000',
 '100m',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '10s',
 '10th',
 '11',
 '110',
 '111',
 '112',
 '113',
 '1138',
 '114',
 '115',
 '117',
 '118',
 '11th',
 '12',
 '121',
 '122',
 '123',
 '125',
 '126',
 '127',
 '1272',
 '128',
 '129',
 '1298',
 '13',
 '130',
 '1305',
 '132',
 '133',
 '135',
 '137',
 '138',
 '13th',
 '14',
 '143',
 '14th',
 '15',
 '150',
 '1500s',
 '151',
 '152',
 '153',
 '155',
 '1583',
 '15th',
 '16',
 '160',
 '1600',
 '1600s',
 '161',
 '167',
 '1692',
 '16mm',
 '16x9',
 '17',
 '170',
 '1700s',
 '1709',
 '172',
 '175',
 '1773',
 '1791',
 '1792',
 '1793',
 '1794',
 '1799',
 '17th',
 '18',
 '180',
 '1800',
 '1800s',
 '1812',
 '1830s',
 '1839',
 '1847',
 '1862',
 '1865',
 '1869',
 '1871',
 '1885',
 '1888',
 '189',
 '1896',
 '1898',
 '18s',
 '18th',
 '19',
 '1900',
 '1900s',
 '1903',
 '1912',
 '1914',
 '1916',
 '1919',
 '1920s',
 '1922'

In [64]:
# transform training data into a "document-term matrix'
X_train_dtm = vect.transform(X_train)
X_train_dtm

<1500x35152 sparse matrix of type '<class 'numpy.int64'>'
	with 495843 stored elements in Compressed Sparse Row format>

In [65]:
# convert sparse matrix to a dense matrix
X_train_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [66]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names())

,00,000,0009f,007,00s,03,04,05,05425,10,...,zucker,zuehlke,zuko,zulu,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,8,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<500x35152 sparse matrix of type '<class 'numpy.int64'>'
	with 165969 stored elements in Compressed Sparse Row format>

## Class prediction with Multinomial Naive Bayes

In [68]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [69]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

ValueError: Unknown label type: (array([1, -1, 1, ..., -1, 1, -1], dtype=object),)

In [ ]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)
y_pred_class

### Performance evaluation

In [ ]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

In [ ]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

In [ ]:
# print review for the false positives
X_test[(y_pred_class==1) & (y_test==-1)]

In [ ]:
# print review for the false negatives
X_test[(y_pred_class==-1) & (y_test==1)]

In [ ]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

## Class prediction with 